<a href="https://colab.research.google.com/github/saisriy/ShroudX/blob/main/audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
from scipy.io import wavfile
from scipy.fft import fft, ifft
from scipy.signal import resample


sample_rate, data = wavfile.read('manideep.wav')


if data.ndim > 1:
    data = data.mean(axis=1)

# Apply FFT to convert to frequency domain
freq_data = fft(data)

# Calculate the magnitude spectrum
magnitude_spectrum = np.abs(freq_data)

# Define a threshold to select dominant frequencies (adjust percentile as needed)
threshold = np.percentile(magnitude_spectrum, 90)

# Create a mask for dominant frequencies
mask = magnitude_spectrum > threshold

# Apply the mask to the frequency data
filtered_freq_data = freq_data * mask

# Perform IFFT to reconstruct the audio
reconstructed_audio = np.real(ifft(filtered_freq_data))

# Downsample the reconstructed audio
downsampling_factor = 20  # Adjust as needed
downsampled_audio = resample(reconstructed_audio, int(len(reconstructed_audio) / downsampling_factor))

# Update 'love' with the downsampled audio
love = np.array(downsampled_audio)

# Normalize 'love' to 16-bit PCM range
love = np.int16(love / np.max(np.abs(love)) * 32767)
love = love / np.max(np.abs(love))  # Scale to [-1, 1]
love = (love + 1) * 127.5            # Shift to [0, 255]
love = np.uint8(love)
simple = love
print(simple)
# np.savetxt('simple_audio.txt', simple, fmt='%d')

audio_string = " ".join(map(str, simple))
with open("encodable_audio_text.txt", "w") as f:
    f.write(audio_string)
# Write the downsampled and normalized 'love' to a new WAV file
wavfile.write('reconstructed_audio_downsampled.wav', int(sample_rate / downsampling_factor), love)

print(love.shape) # Print the shape of 'love' to verify the reduction

[124 130 126 ... 127 128 126]
(6758,)


In [49]:
import cv2
import numpy as np
import secrets
import time
from skimage.metrics import structural_similarity as ssim
import os

# === EMBED ===
def embed_text_random_expire(image_path, text_file, output_path, expiry_seconds):
    def generate_key():
        return secrets.randbits(32)

    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image at path '{image_path}'.")
        return None

    # Read numbers from file (e.g., 8-bit ints)
    with open(text_file, "r", encoding="utf-8") as file:
        message = list(map(int, file.read().strip().split()))

    # Add timestamp and expiry
    timestamp_bin = format(int(time.time()), '032b')
    expiry_bin = format(int(expiry_seconds), '032b')

    # Use five 255s as a unique end marker
    end_marker = [255, 255, 255, 255, 255]
    message += end_marker

    # Convert all to binary
    binary_message = timestamp_bin + expiry_bin + ''.join(format(num, '08b') for num in message)

    # Generate random pixel indices
    key = generate_key()
    np.random.seed(key)
    pixel_indices = np.random.choice(image.size // 3, len(binary_message), replace=False)

    for i, bit in enumerate(binary_message):
        idx = pixel_indices[i]
        row = idx // image.shape[1]
        col = idx % image.shape[1]
        channel = i % 3
        image[row, col, channel] = (image[row, col, channel] & 254) | int(bit)

    cv2.imwrite(output_path, image)
    print(f"Message embedded successfully into {output_path}")
    print(f"Key: {key}")
    return key

# === EXTRACT ===
def extract_text_random_expire(image_path, output_text_file, key):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image at path '{image_path}'.")
        return False

    np.random.seed(key)
    pixel_indices = np.random.choice(image.size // 3, image.size // 3, replace=False)

    binary_bits = []
    for i in pixel_indices:
        row = i // image.shape[1]
        col = i % image.shape[1]
        channel = len(binary_bits) % 3
        bit = image[row, col, channel] & 1
        binary_bits.append(str(bit))

        # Check for end marker every 8 bits
        if len(binary_bits) >= 40 and len(binary_bits) % 8 == 0:
            last_five = [
                int(''.join(binary_bits[-8 * i: -8 * (i - 1) if i > 1 else None]), 2)
                for i in range(1, 6)
            ]
            if last_five == [255, 255, 255, 255, 255]:
                break
    else:
        print("Error: End marker not found. Possibly wrong key or corrupted data.")
        return False

    binary_message = ''.join(binary_bits[:-40])  # Remove end marker

    try:
        timestamp = int(binary_message[:32], 2)
        expiry = int(binary_message[32:64], 2)
        current = int(time.time())

        if current - timestamp > expiry:
            print("Error: Message has expired and cannot be extracted.")
            return False

        binary_message = binary_message[64:]
        message = [str(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message), 8)]

    except Exception as e:
        print(f"Error decoding message: {e}")
        return False

    with open(output_text_file, "w", encoding="utf-8") as file:
        file.write(" ".join(message))

    print(f"Message extracted and saved to {output_text_file}")
    return True

# === METRICS ===
def compute_psnr(original, stego):
    mse = np.mean((original - stego) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

def compute_ssim(original, stego):
    return ssim(original, stego, data_range=255, channel_axis=-1)

def compute_hamming_distance(original, stego):
    return np.sum(np.unpackbits(original.flatten()) != np.unpackbits(stego.flatten()))

def evaluate_steganography(original_path, stego_path):
    original = cv2.imread(original_path)
    stego = cv2.imread(stego_path)
    if original is None or stego is None:
        print("Error: Could not read one of the images.")
        return

    psnr_val = compute_psnr(original, stego)
    ssim_val = compute_ssim(cv2.cvtColor(original, cv2.COLOR_BGR2GRAY), cv2.cvtColor(stego, cv2.COLOR_BGR2GRAY))
    hamming_val = compute_hamming_distance(original, stego)

    print(f"PSNR: {psnr_val:.2f} dB")
    print(f"SSIM: {ssim_val:.4f}")
    print(f"Hamming Distance: {hamming_val}")

    return {
        "psnr": psnr_val,
        "ssim": ssim_val,
        "hamming": hamming_val
    }

# === RUN PIPELINE ===
original_image = "image240.jpg"
text_file = "encodable_audio_text.txt"
stego_image = "encoded_image240.png"
decoded_text_file = "decodedmad.txt"

if os.path.exists(original_image) and os.path.exists(text_file):
    key = embed_text_random_expire(original_image, text_file, stego_image, expiry_seconds=300)
    if key:
        extract_text_random_expire(stego_image, decoded_text_file, key)
        print("\nEvaluation Metrics:")
        evaluate_steganography(original_image, stego_image)
else:
    print("Make sure your image and text files are uploaded.")


Message embedded successfully into encoded_image240.png
Key: 955103414
Message extracted and saved to decodedmad.txt

Evaluation Metrics:
PSNR: 56.22 dB
SSIM: 0.9982
Hamming Distance: 26807


In [50]:
def decode_text_to_array(decoded_text_file):
    # Read the decoded text file
    with open(decoded_text_file, "r", encoding="utf-8") as file:
        audio_string = file.read()

    # Convert the string back to a list of integers
    # Assuming it was originally space-separated integers
    audio_array = list(map(int, audio_string.split()))

    return audio_array

In [51]:
decoded_text_file = "decoded_audio.txt"
audio_array =np.array(decode_text_to_array(decoded_text_file))
print(audio_array)
print(audio_array.shape)

[124 130 126 ... 127 128 126]
(6758,)


In [52]:
audio_array = np.uint8(audio_array)  # If audio_array is not already int16
wavfile.write('decodedmad.wav', int(sample_rate / downsampling_factor), audio_array)

In [11]:
def are_files_identical(file1_path, file2_path):
    try:
        with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
            # Read both files
            file1_content = file1.read()
            file2_content = file2.read()

            # Compare the contents
            if file1_content == file2_content:
                print("The files are identical.")
                return True
            else:
                print("The files are not identical.")
                return False

    except FileNotFoundError as e:
        print(f"Error: {e}")
        return False

# Example usage:
file1 = "encodable_audio_text.txt"
file2 = "decoded_audio.txt"
are_files_identical(file1, file2)


The files are identical.


True